# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

## Load the Data

Start by importing the data from the pickle file.

In [2]:
# TODO: Implement load the data here.
with open('train.p', 'rb') as f:
    data = pickle.load(f)

In [10]:
data.keys()

dict_keys(['labels', 'features'])

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [23]:
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)

In [7]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [24]:
X_train.shape

(29406, 32, 32, 3)

In [25]:
type(X_train)

numpy.ndarray

In [27]:
X_train[1]

array([[[ 76,  89, 111],
        [ 75,  90, 110],
        [ 75,  88, 109],
        ..., 
        [ 76,  88, 109],
        [ 75,  88, 108],
        [ 76,  87, 106]],

       [[ 73,  87, 107],
        [ 77,  88, 108],
        [ 74,  85, 105],
        ..., 
        [ 75,  89, 109],
        [ 74,  89, 110],
        [ 75,  88, 109]],

       [[ 74,  85, 103],
        [ 74,  85, 103],
        [ 70,  81, 101],
        ..., 
        [ 75,  89, 107],
        [ 75,  89, 109],
        [ 74,  87, 109]],

       ..., 
       [[ 81,  95, 116],
        [ 79,  92, 113],
        [ 79,  90, 110],
        ..., 
        [ 76,  91, 112],
        [ 78,  89, 109],
        [ 75,  88, 107]],

       [[ 78,  90, 110],
        [ 82,  95, 115],
        [ 77,  87, 108],
        ..., 
        [ 76,  90, 112],
        [ 79,  91, 110],
        [ 78,  90, 108]],

       [[ 75,  86, 110],
        [ 76,  85,  99],
        [ 49,  54,  60],
        ..., 
        [ 73,  84,  98],
        [ 64,  72,  85],
        [ 64,  79,

In [28]:
X_train[1][0]

array([[ 76,  89, 111],
       [ 75,  90, 110],
       [ 75,  88, 109],
       [ 74,  85, 106],
       [ 73,  84, 105],
       [ 74,  85, 110],
       [ 74,  87, 110],
       [ 75,  85, 101],
       [ 70,  81,  94],
       [ 66,  79,  98],
       [ 70,  83,  99],
       [ 69,  80,  94],
       [ 69,  80,  96],
       [ 67,  81,  99],
       [ 72,  87, 106],
       [ 76,  86, 104],
       [ 72,  79,  96],
       [ 62,  76,  93],
       [ 68,  82,  98],
       [ 71,  81,  98],
       [ 70,  81,  98],
       [ 69,  83, 101],
       [ 72,  86, 108],
       [ 75,  88, 108],
       [ 72,  86, 104],
       [ 71,  84, 102],
       [ 71,  85, 104],
       [ 74,  88, 107],
       [ 75,  88, 109],
       [ 76,  88, 109],
       [ 75,  88, 108],
       [ 76,  87, 106]], dtype=uint8)

In [29]:
X_train[1][0][0]

array([ 76,  89, 111], dtype=uint8)

In [34]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)

In [35]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [ ]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Dense, Input, Activation

In [40]:
model = Sequential()
model.add(Dense(128, input_shape=(32*32*3,)))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [41]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [42]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_5 (None, 3072) (None, 128) <function linear at 0x7f328ce66d90>
activation_3 (None, 128) (None, 128) <function relu at 0x7f328ce66b70>
dense_6 (None, 128) (None, 43) <function linear at 0x7f328ce66d90>
activation_4 (None, 43) (None, 43) <function softmax at 0x7f328ce66950>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [43]:
# TODO: Compile and train the model here.
from keras.utils import np_utils

# One-hot encode the labels
Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)

# Reshapen die inputten
X_train_flat = X_train.reshape(-1, 32*32*3)
X_val_flat = X_val.reshape(-1, 32*32*3)

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train_flat, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val_flat, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 128)           393344      dense_input_3[0][0]              
____________________________________________________________________________________________________
activation_3 (Activation)        (None, 128)           0           dense_5[0][0]                    
____________________________________________________________________________________________________
dense_6 (Dense)                  (None, 43)            5547        activation_3[0][0]               
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 43)            0           dense_6[0][0]                    
Total params: 398891
______________________________________________________________________

In [44]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
94.19%

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [45]:
## Reload and re-normalize the data
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [46]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

# TODO: Compile and train the model here.

from keras.layers import Conv2D, Flatten

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)


model = Sequential()
# Note that Conv2D defaults to 'valid' padding
model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 28800)         0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 128)           3686528     flatten_1[0][0]                  
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 43)            5547        dense_7[0][0]                    
Total params: 3692971
_____________________________________________________________________

In [47]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
97.39%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [49]:
## Reload and re-normalize the data
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [50]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

# TODO: Compile and train the model here.
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)


model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_2 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 15, 15, 32)    0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 7200)          0           activation_5[0][0]               
___________________________________________________________________________________________

In [51]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
98.21%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [52]:
## Reload and re-normalize the data
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [53]:
# TODO: Re-construct the network and add dropout after the pooling layer.

# TODO: Compile and train the model here.
from keras.layers import Dropout

model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2,2)))
model.add((Dropout(0.5)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_2[0][0]             
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 15, 15, 32)    0           dropout_1[0][0]                  
___________________________________________________________________________________________

In [54]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
98.25%

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:**  
98.25%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [55]:
# TODO: Load test data
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
# TODO: Preprocess data & one-hot encode the labels
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

# TODO: Evaluate model on test data
model.evaluate(X_test, Y_test)

12608/12630 [============================>.] - ETA: 0s

[0.4935946452191729, 0.91163895483160429]

**Test Accuracy:**
91.16%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.